In [1]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D,Bidirectional
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

C:\Users\Tanvir\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv("dataprep.csv")  

In [27]:
df.Label.value_counts()

1    15075
0     5285
Name: Label, dtype: int64

In [3]:
df['Label'] = df.Label.map({'H':0, 'N':1})

In [8]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        #review = re.sub(r'^@', '', review, flags=re.MULTILINE)
        #review=re.sub(r'[^a-zA-Z]','',review)
        #review = re.sub(r'^https?:\/\/.*[\r\n]*', '', review, flags=re.MULTILINE)
        review = re.sub(r'RT', '', review, flags=re.MULTILINE)
        #' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)','',review).split())
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        #review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review = re.sub(r"(?:\@|https?\://)\S+", "", review)
        #review=''.join(review)
        review_corpus.append(review)
    return review_corpus

In [9]:
df['clean_review']=clean_review(df.Statement.values)
df.head()

Label                                          Statement  \
0      0  @Grumpy_P_Sloth @deanesmay feminists argue for...   
1      0  1948Army of England helped the Jews to occupy ...   
2      0  Dutch Leader Says Europe to collapse In 6 Week...   
3      0  RT @__DeLay: The next day the Romans and the J...   
4      0  RT @Bakersman_Joe: When Hitler Invited The Jew...   

                                        clean_review  
0    feminists argue for raising minimum wage cau...  
1  1948Army of England helped the Jews to occupy ...  
2  Dutch Leader Says Europe to collapse In 6 Week...  
3    The next day the Romans and the Jews or wate...  
4    When Hitler Invited The Jews To The Smoke Se...

In [10]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['clean_review'].values)
X = tokenizer.texts_to_sequences(df['clean_review'].values)
X = pad_sequences(X)

In [18]:
embed_dim = 128
lstm_out = 196

from keras.layers import Bidirectional,GRU

model = Sequential()

model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(LSTM(128,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(64,return_sequences=True))
model.add(Bidirectional(GRU(lstm_out, return_sequences=True)))
#model.add(Bidirectional(LSTM(lstm_out, return_sequences=True)))
model.add(Bidirectional(GRU(lstm_out, recurrent_dropout=0.2, dropout=0.2, return_sequences=False)))
#model.add(Bidirectional(LSTM(lstm_out, recurrent_dropout=0.2, dropout=0.2, return_sequences=False)))

model.add(Dense(2,activation='sigmoid',kernel_initializer='TruncatedNormal'))
model.compile(loss = 'categorical_crossentropy', optimizer='Adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 38, 128)           256000    
_________________________________________________________________
lstm_21 (LSTM)               (None, 38, 128)           131584    
_________________________________________________________________
lstm_22 (LSTM)               (None, 38, 128)           131584    
_________________________________________________________________
lstm_23 (LSTM)               (None, 38, 128)           131584    
_________________________________________________________________
lstm_24 (LSTM)               (None, 38, 128)           131584    
_________________________________________________________________
lstm_25 (LSTM)               (None, 38, 64)            49408     
_________________________________________________________________
lstm_26 (LSTM)               (None, 38, 64)            33024     
__________

In [19]:
Y = pd.get_dummies(df['Label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(16288, 38) (16288, 2)
(4072, 38) (4072, 2)


In [20]:
X_val = X_train[:4072]
Y_val = Y_train[:4072]

In [21]:
partial_X_train = X_train[4072:]
partial_Y_train = Y_train[4072:]

In [25]:
batch_size = 64
history = model.fit(X_train, 
                    Y_train, 
                    epochs = 50, 
                    batch_size=batch_size, 
                    validation_data=(X_val, Y_val))

Train on 16288 samples, validate on 4072 samples
Epoch 1/50
16288/16288 [==============================] - 918s 56ms/step - loss: 0.2240 - acc: 0.9040 - val_loss: 0.1689 - val_acc: 0.9352
Epoch 2/50
16288/16288 [==============================] - 823s 51ms/step - loss: 0.2244 - acc: 0.9035 - val_loss: 0.1603 - val_acc: 0.9381
Epoch 3/50
16288/16288 [==============================] - 812s 50ms/step - loss: 0.2161 - acc: 0.9072 - val_loss: 0.1739 - val_acc: 0.9361
Epoch 4/50
16288/16288 [==============================] - 813s 50ms/step - loss: 0.2149 - acc: 0.9082 - val_loss: 0.1579 - val_acc: 0.9398
Epoch 5/50
16288/16288 [==============================] - 824s 51ms/step - loss: 0.2083 - acc: 0.9128 - val_loss: 0.1534 - val_acc: 0.9430
Epoch 6/50
16288/16288 [==============================] - 817s 50ms/step - loss: 0.2001 - acc: 0.9155 - val_loss: 0.1412 - val_acc: 0.9430
Epoch 7/50
16288/16288 [==============================] - 805s 49ms/step - loss: 0.1957 - acc: 0.9188 - val_loss: 0.1

In [26]:
loss, accuracy = model.evaluate(X_train,Y_train , verbose=False)
print("Training Accuracy: {:.2f}".format(accuracy))
loss, accuracy = model.evaluate(X_test,Y_test, verbose=False)
print("Testing Accuracy:  {:.2f}".format(accuracy))

Training Accuracy: 0.97
Testing Accuracy:  0.68
